In [1]:

%load_ext autoreload
%autoreload 2

In [ ]:
#Get modules
#indicators
from Incision import *
from Headcuts import *
from Erosion import *
from Invasives import *
from Bioassessment import *
from Conifer import *
from VegVigor import *
from AOP import *
from Habitat import *
from Ditches import *


# Process Indicators Post To Staging Tables

## Process Every Year= In-house Monitoring Data

In [ ]:
#CONIFER#
#!!!No Need to update if there is no new information
#Current code grabs old data which is already in staging table
from Conifer import *
df=get_conifersez_data_sql()
readydf=process_conifer(df)
post_conifer_data(df, draft=True)

In [7]:
#AOP#
#!!! No need to update if there is no new information- 
# code isn't complete for new data- do manually for now until we set up a script for sde.aquaticorganismpassage_usfs
#Current code grabs old data which is already in staging table
from AOP import *
#Get data from SEZ_Assessment Unit Table in sde (this indicator doesn't get updated much)
df = get_aopsez_data_sql()
#rename the sql data so that we can 
readydf = Process_aop(df)
#post_AOP_data(readydf, draft=True)

In [8]:
#BIOASSESSMENT#done Update EVERY YEAR
#Last updated 2023
from Bioassessment import *
# from Bioassessment import *
df = get_bioassessment_data()
#clean, process, grade data
readydf = process_grade_bioassessment(df, 2023)
#post data to sde.VECTOR.stage headcut table or csv
post_bioassessment(readydf, draft=True)


In [ ]:
#INVASIVE SPECIES#done Update EVERY YEAR
#Last Updated 2023
from Invasives import *
#Get USFS rest service data spatially joined to SEZ's
usfsdf= get_USFSinvasive_data()
#Get new invasive data from sde collect
Idf = get_combined_survey_and_invasive_data()
#clean, merge with external data/ format plant type names
df = merge_format_prioritize_invasive(Idf, usfsdf, 2024)
#Process Data and grade/rate assessment
#Be sure to update the priority list in "F:\GIS\PROJECTS\ResearchAnalysis\SEZ\Invasives Priority Lookup.csv"
#rate_invasives is in utils if you need to update that
invasive_summary, invasive_priority_summary = process_grade_invasives(df)
#Format Data so it is ready to be added to sde/csv for one last QA
readydf= final_format_invasive(df, invasive_priority_summary)
#post data to sde.VECTOR.stage headcut table or csv
post_invasive(readydf, draft=True)

In [ ]:
invasive_priority_summary.info()

In [ ]:
#HEADCUT#done#Update Every Year
from Headcuts import *
#Get raw headcut data prior 2024 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allheadcut_data()
#Get new headcut data from sde collect
df = get_combined_survey_and_headcut_data()
#clean, process, grade data
readydf = process_grade_headcut(df, 2024)
#post data to sde.VECTOR.stage headcut table or csv
post_headcut(readydf, draft=True)


In [50]:
#INCISION# Update Every Year
#Get raw incision data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
#df= get_allincision_data()
#Get incision data from sde collect
df = get_combined_survey_and_incision_data()
#clean, process, grade data
readydf = process_grade_incision(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_incision(readydf, draft=True)

In [ ]:
#EROSION# Update Every Year
from Erosion import *

#Get raw erosion data prior 2023 and earlier that is stored in gdbs in F:GIS/data/Monitoring/
df= get_erosion_data()

#clean, process, grade data
readydf = process_grade_erosion(df, 2024)
#post data to sde.VECTOR.stage incision table or csv
post_erosion(readydf, draft=True)

In [ ]:
#VEG VIGOR#
#Use data from SEZTable until we get new lidar data
df=get_sez_data_sql()
#df= get_oldvegvig_data()
post_Veg_data(df, draft=True)


# Staging Tables to Final Table

In [16]:
from functools import reduce
from FormatStagingTables import *
# Staging to final
dfbanks = get_fs_data(bank_stability_url)
dfbiotic = get_fs_data(biotic_integrity_url)
dfconifer = get_fs_data(conifer_url)
dfditch = get_fs_data(ditches_url)
dfinvasive = get_fs_data(invasives_url)
dfhabitat = get_fs_data(Hab_Frag_url)
dfvegetation = get_fs_data(vegetation_url)
dfincision = get_fs_data(incision_url)
dfheadcuts = get_fs_data(headcuts_url)
dfAOP = get_fs_data(AOP_url)
dfSEZ = get_fs_data_spatial(SEZ_url)
dfheadcuts.drop(columns=['small', 'medium', 'large'], inplace=True)
#Format Staging Tables
#format biotic data average if two streams in one assessment unit
averaged_biotic_df=average_biotic_scores(dfbiotic)
#Prep SEZ Baseline Data for assessment unit...will need to rethink if acreage changes.. or just manually change in sde
keep_columns = ['SHAPE', 'SEZ_ID', 'Feature_Type', 'SEZ_Type', 'Ownership_Primary', 'Ownership_Secondary', 'Ownership_Secondary_2', 'Ownership_Secondary_3', 'Acres', 'Comments']
#dfSEZ is assessment unit information from SDE?
dfSEZ.loc[:,keep_columns].copy()
dfSEZ['SEZ_ID']= dfSEZ['SEZ_ID'].astype(int)
#Same for meadow(large polygon) and riverine(small polygon) data drop these columns because not needed in final merge, will assign SEZ ID later
columns_to_drop = {'Year', 'SEZ_ID', 'GlobalID', 'last_edited_user', 'created_date', 'OBJECTID', 'created_user', 'last_edited_date'}

#Name dataframes so we can reference later
largepolygondata= {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}


#Staging Tables Riverine/ small polygons
smallpolygondata = {'dfbanks': dfbanks, 
                'dfaveraged_biotic': averaged_biotic_df,
                'dfconifer': dfconifer,
                'dfditch': dfditch,
                'dfinvasive': dfinvasive,
                'dfhabitat': dfhabitat,
                'dfvegetation': dfvegetation,
                'dfincision': dfincision,
                'dfheadcuts': dfheadcuts,
                'dfAOP': dfAOP
}
# Process large polygon (meadow) and small polygon (riverine) data -
#Get most recent scores for each indicator
# Clean Data in each dataframe, assign SEZ ID to each assessment unit, add year to datasource column for formatting purposes
processed_largepolygon_data = process_data(largepolygondata, lookup_dict, columns_to_drop)
processed_smallpolygon_data = process_data(smallpolygondata, lookup_riverine, columns_to_drop)

# Merge large polygon and small polygon data
# Function to merge all DataFrames on multiple keys(why on multiple keys? and not just SEZ ID)
#moved to formatstagingtables.py
# def merge_dataframes(data_dict, keys):
 #   return reduce(lambda left, right: pd.merge(left, right, on=keys, how='outer'), data_dict.values())

# Merge small polygon DataFrames
smallpolygon_df = merge_dataframes(processed_smallpolygon_data)

# Merge large polygon DataFrames so all indicators that have data
largepolygon_df = merge_dataframes(processed_largepolygon_data)

# Append smallpolygon_df to largepolygon_df to have a complete list of all meadows and all sez ids
final_combined_df = pd.concat([largepolygon_df, smallpolygon_df], ignore_index=True)

# Print the final combined DataFrame to check
print("Final Combined DataFrame:")
print(final_combined_df)



c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\FormatStagingTables.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col] + ', ' + df['Year'].astype(str)
c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\FormatStagingTables.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col] + ', ' + df['Year'].astype(str)
c:\Users\snewsome\Documents\GitHub\Monitoring\SEZ_scripts\FormatStagingTables.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of

ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().